In [1]:
import pandas as pd
import numpy as np
import re
import os
import utils 
import string

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [2]:
data_dir = "data/"

files = ["H-1B_Disclosure_Data_FY16.xlsx", 
            "H-1B_Disclosure_Data_FY15_Q4.xlsx", 
            "H-1B_Disclosure_Data_FY17.xlsx"]



In [3]:
data = pd.read_excel(data_dir + files[2])

In [4]:
columns = list(data)
print(columns)
print(data.dtypes)


['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE', 'VISA_CLASS', 'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE', 'EMPLOYER_NAME', 'EMPLOYER_ADDRESS', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE', 'EMPLOYER_COUNTRY', 'EMPLOYER_PROVINCE', 'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT', 'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE', 'JOB_TITLE', 'SOC_CODE', 'SOC_NAME', 'NAICS_CODE', 'TOTAL_WORKERS', 'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY', 'PW_SOURCE', 'PW_SOURCE_YEAR', 'PW_SOURCE_OTHER', 'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY', 'H-1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'WORKSITE_CITY', 'WORKSITE_COUNTY', 'WORKSITE_STATE', 'WORKSITE_POSTAL_CODE', 'ORIGINAL_CERT_DATE']
CASE_NUMBER                      object
CASE_STATUS                      object
CASE_SUBMITTED           datetime64[ns]
DECISION_DATE            datetime64[ns]
VISA_CLASS                       object
EMPLOYMENT_START_DATE    datetime64[ns]
EM

In [5]:
data.head()

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,AGENT_ATTORNEY_NAME,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,JOB_TITLE,SOC_CODE,SOC_NAME,NAICS_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_SOURCE,PW_SOURCE_YEAR,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE
0,I-200-16055-173457,CERTIFIED-WITHDRAWN,2016-02-24,2016-10-01,H-1B,2016-08-10,2019-08-10,DISCOVER PRODUCTS INC.,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,UNITED STATES OF AMERICA,NaN,2244050900,NaN,"ELLSWORTH, CHAD",NEW YORK,NY,ASSOCIATE DATA INTEGRATION,15-1121,COMPUTER SYSTEMS ANALYSTS,522210.0,1,Y,59197.0,Year,OES,2015.0,OFLC ONLINE DATA CENTER,65811.0,67320.0,Year,N,N,RIVERWOODS,LAKE,IL,60015,2016-03-01
1,I-200-16064-557834,CERTIFIED-WITHDRAWN,2016-03-04,2016-10-01,H-1B,2016-08-16,2019-08-16,DFS SERVICES LLC,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,UNITED STATES OF AMERICA,NaN,2244050900,NaN,"ELLSWORTH, CHAD",NEW YORK,NY,SENIOR ASSOCIATE,15-2031,OPERATIONS RESEARCH ANALYSTS,522210.0,1,Y,49800.0,Year,Other,2015.0,TOWERS WATSON DATA SERVICES 2015 CSR PROFESSIO...,53000.0,57200.0,Year,N,N,RIVERWOODS,LAKE,IL,60015,2016-03-08
2,I-200-16063-996093,CERTIFIED-WITHDRAWN,2016-03-10,2016-10-01,H-1B,2016-09-09,2019-09-09,EASTBANC TECHNOLOGIES LLC,1211 31ST ST. NW,WASHINGTON,DC,20007,UNITED STATES OF AMERICA,NaN,2022953000,NaN,"BURKE, KAREN",WASHINGTON,DC,.NET SOFTWARE PROGRAMMER,15-1131,COMPUTER PROGRAMMERS,541511.0,2,Y,76502.0,Year,OES,2015.0,OFLC ONLINE DATA CENTER,77000.0,0.0,Year,Y,N,WASHINGTON,NaN,DC,20007,2016-03-16
3,I-200-16272-196340,WITHDRAWN,2016-09-28,2016-10-01,H-1B,2017-01-26,2020-01-25,INFO SERVICES LLC,17177 NORTH LAUREL PARK DR,LIVONIA,MI,48152,UNITED STATES OF AMERICA,NaN,7343776007,NaN,",",NaN,NaN,PROJECT MANAGER,15-1199,"COMPUTER OCCUPATIONS, ALL OTHER",541511.0,1,Y,90376.0,Year,OES,2016.0,OFLC ONLINE DATA CENTER,102000.0,0.0,Year,Y,N,JERSEY CITY,HUDSON,NJ,07302,NaT
4,I-200-15053-636744,CERTIFIED-WITHDRAWN,2015-02-22,2016-10-02,H-1B,2015-03-01,2018-03-01,BB&T CORPORATION,223 WEST NASH STREET,WILSON,NC,27893,UNITED STATES OF AMERICA,NaN,2522462031,NaN,"SCOFIELD, EILEEN",ATLANTA,GA,ASSOCIATE - ESOTERIC ASSET BACKED SECURITIES,13-2041,CREDIT ANALYSTS,522110.0,1,Y,116605.0,Year,OES,2015.0,OFLC ONLINE DATA CENTER,132500.0,0.0,Year,N,N,NEW YORK,NEW YORK,NY,10036,2015-02-26


In [6]:
print(len(data))

528146


In [7]:
print(len(data.CASE_NUMBER.unique()))

528146


In [8]:
print(data.CASE_STATUS.unique())

['CERTIFIED-WITHDRAWN' 'WITHDRAWN' 'CERTIFIED' 'DENIED']


In [9]:
print(data.VISA_CLASS.unique())

['H-1B' 'E-3 Australian' 'H-1B1 Singapore' 'H-1B1 Chile']


In [10]:
data.CASE_SUBMITTED.isnull().sum()

0

In [11]:
data.DECISION_DATE.isnull().sum()

0

In [12]:
data['CASE_SUBMITTED'] = data['CASE_SUBMITTED'].dt.strftime('%Y-%m-%d')
data['DECISION_DATE'] = data['DECISION_DATE'].dt.strftime('%Y-%m-%d')

In [13]:
data.EMPLOYER_ADDRESS.isnull().sum()

6

In [14]:
data['EMPLOYER_ADDRESS'] = data['EMPLOYER_ADDRESS'].apply(utils.emp_address)

In [15]:
data.EMPLOYER_POSTAL_CODE.isnull().sum()

15

In [16]:
data.EMPLOYER_POSTAL_CODE = data['EMPLOYER_POSTAL_CODE'].apply(utils.update_fix_zip)

Not a number:  L0S ANGELES
zip code error:  300341
Not a number:  CA
Not a number:  V6J 1C7
Not a number:  V6J 1C7
zip code error:  882017630014
zip code error:  926002
Not a number:  V6J 1C7
Not a number:  NEW Y0RK
Not a number:  V6J 1C7
zip code error:  372075
zip code error:  752147
Not a number:  CA
zip code error:  440077
Not a number:  TINIAN
zip code error:  372075
Not a number:  0T65208224
Not a number:  V6J 1C7
zip code error:  238220
zip code error:  9
zip code error:  541512
Not a number:  nan
Not a number:  SAN FRANCISC0
zip code error:  926141
Not a number:  nan
Not a number:  GARDEN GR0VE
Not a number:  W00DCLIFF LAKE
zip code error:  511210
zip code error:  9739090360
zip code error:  080021
Not a number:  TARRANT
zip code error:  787558
zip code error:  9739090360
zip code error:  484369
Not a number:  CA
Not a number:  nan
Not a number:  V6J 1C7
zip code error:  675205
Not a number:  SAN FRANCISC0
Not a number:  NEW Y0RK
Not a number:  NY
zip code error:  088902
Not a 

In [17]:
data.EMPLOYER_POSTAL_CODE.isnull().sum()

111

In [18]:
data.WORKSITE_POSTAL_CODE.isnull().sum()

15

In [19]:
data.WORKSITE_POSTAL_CODE = data['WORKSITE_POSTAL_CODE'].apply(utils.update_fix_zip)

zip code error:  981109
zip code error:  799103
zip code error:  553443
zip code error:  951354
zip code error:  101676
zip code error:  927702
zip code error:  617913
zip code error:  110167
zip code error:  121244
zip code error:  914133
Not a number:  ARLINGT0N
zip code error:  198014
zip code error:  680892
zip code error:  100036
Not a number:  HARRIS
Not a number:  CUYAH0GA
zip code error:  483226
zip code error:  100036
zip code error:  275602
Not a number:  TIPPECAN0E
Not a number:  nan
zip code error:  4630100
zip code error:  5457900
zip code error:  115627
zip code error:  0853676155
zip code error:  030328
zip code error:  900015
zip code error:  030303
zip code error:  0853676155
zip code error:  060606
zip code error:  800528
zip code error:  0853676155
zip code error:  932547
Not a number:  DALLAS
zip code error:  018861
zip code error:  156444
zip code error:  106101
zip code error:  100013
zip code error:  9410509200
Not a number:  HILLSB0R0UGH
Not a number:  DALLAS
zi

In [20]:
data.WORKSITE_POSTAL_CODE.isnull().sum()

291

In [21]:
data["EMPLOYER_CITY"]=data.EMPLOYER_CITY.apply(utils.uppercase_nopunct)
data["EMPLOYER_STATE"]=data.EMPLOYER_STATE.apply(utils.uppercase_nopunct)
data["WORKSITE_CITY"]=data.WORKSITE_CITY.apply(utils.uppercase_nopunct)
data["WORKSITE_STATE"]=data.WORKSITE_STATE.apply(utils.uppercase_nopunct)

In [23]:
data[data['WORKSITE_STATE']=='PW']

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,AGENT_ATTORNEY_NAME,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,JOB_TITLE,SOC_CODE,SOC_NAME,NAICS_CODE,TOTAL_WORKERS,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_SOURCE,PW_SOURCE_YEAR,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H-1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE
67138,I-200-16336-545842,CERTIFIED,2016-12-01,2016-12-07,H-1B,2016-12-05,2019-12-04,BI LABS INC,517 ROUTE 1 S,ISELIN,NJ,08830,UNITED STATES OF AMERICA,NaN,7326364201,NaN,",",NaN,NaN,PROGRAMMER ANALYST,15-1131,COMPUTER PROGRAMMERS,541511.0,1,Y,55349.0,Year,OES,2016.0,OFLC ONLINE DATA CENTER,60000.0,0.0,Year,N,N,PLYMOUTH MEETING,MONTGOMERY,PW,19462,NaT


In [24]:
data.WORKSITE_STATE = data.WORKSITE_STATE.replace('PW', 'PA')

In [25]:
data.EMPLOYER_STATE.isnull().sum()

15

In [26]:
data.EMPLOYER_STATE = data[['CASE_NUMBER','EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE']].apply(utils.update_fix_states, axis =1)

Old state was FM and new state is FL
Old state was nan and new state is CA Case Number: I-200-16306-591423
Old state was AS and new state is WA
Old state was nan and new state is MA Case Number: I-200-17059-270290
Old state was nan and new state is MA Case Number: I-200-17062-703263


In [27]:
data.EMPLOYER_STATE.isnull().sum()

12

In [28]:
data.WORKSITE_STATE.isnull().sum()

7

In [29]:
data.WORKSITE_STATE = data[['CASE_NUMBER','WORKSITE_STATE', 'WORKSITE_POSTAL_CODE']].apply(utils.worksite_fix_states, axis =1)

Old state was nan and new state is DE Case Number: I-200-16301-945047
Old state was nan and new state is IN Case Number: I-200-17034-707367
Old state was FM and new state is FL Case Number: I-200-17048-549460
Old state was nan and new state is CA Case Number: I-200-17061-694427


In [30]:
data.WORKSITE_STATE.isnull().sum()

4

In [31]:
data.JOB_TITLE = data['JOB_TITLE'].apply(utils.emp_address)

In [32]:
 data['DOT_CODE'] = None

In [33]:
 data.SOC_CODE = data.SOC_CODE.apply(utils.update_fix_socCode)

SOC code error:  113-1161 | 113-1161
SOC code error:  15-119.08 | 15-119.08
SOC code error:  030 | 030
SOC code error:  1-1132 | 1-1132
SOC code error:  15-11212 | 15-11212
SOC code error:  COMPUTER OCCUPATIONS, ALL OTHER | COMPUTEROCCUPATIONS,ALLOTHER
SOC code error:  SOFTWARE DEVELOPERS, APPLICATIONS | SOFTWAREDEVELOPERS,APPLICATIONS
SOC code error:  SOFTWARE DEVELOPERS, APPLICATIONS | SOFTWAREDEVELOPERS,APPLICATIONS
SOC code error:  COMPUTER SYSTEMS ANALYST | COMPUTERSYSTEMSANALYST
SOC code error:  MECHANICAL ENGINEERS | MECHANICALENGINEERS
SOC code error:  SOFTWARE DEVELOPERS, APPLICATIONS | SOFTWAREDEVELOPERS,APPLICATIONS
SOC code error:  ELECTRICAL ENGINEERS | ELECTRICALENGINEERS
SOC code error:  15-119.02 | 15-119.02
SOC code error:  ENGINEERS, ALL OTHER | ENGINEERS,ALLOTHER
SOC code error:  13-161 | 13-161


In [34]:
data.SOC_CODE.unique()

array(['15-1121', '15-2031', '15-1131', '15-1199', '13-2041', '13-2099',
       '13-1111', '13-1161', '15-1132', '19-1029', '25-2031', '19-1021',
       '11-2022', '19-1042', '15-2041', '19-1013', '17-2072', '17-2081',
       '25-9031', '11-3021', '21-1014', '13-2051', '15-1133', '15-1134',
       '11-2021', '17-2071', '17-3011', '27-3042', '19-4021', '25-1066',
       '23-1011', '17-3027', '29-1069', '19-2032', '13-1199', '27-1021',
       '17-2199', '13-1071', '15-1035', '29-1021', '15-1142', '41-3031',
       '13-2011', '15-1141', '41-9031', '29-1061', '19-1099', '17-1012',
       '27-4032', '15-1037', '29-1051', '25-1123', '19-2043', '19-2021',
       '19-2042', '17-2051', '29-1063', '15-1132.00', '15-1143', '17-2011',
       '29-2011', '17-2131', '17-2141', '13-1041', '19-2012', '15-2021',
       '29-1123', '19-4061', '11-3031', '27-3031', '11-9041', '11-9051',
       '25-1054', '17-2031', '15-1111', '17-2112', '15-1122', '25-1022',
       '13-1121', '13-1081', '21-1023', '43-4071

In [35]:
data.TOTAL_WORKERS.isnull().sum()

0

In [36]:
data.EMPLOYER_PHONE = data.EMPLOYER_PHONE.apply(utils.emp_address)

In [37]:
data['EMPLOYER_AREA_CODE'] = [
    str(i)[:3] if len(str(i)) == 10 else np.nan for i in data['EMPLOYER_PHONE']
]

In [38]:
data['WAGE_UNIT_OF_PAY'].unique()

array(['Year', 'Hour', 'Month', 'Bi-Weekly', 'Week', nan], dtype=object)

In [39]:
data['WAGE_UNIT_OF_PAY'] = data['WAGE_UNIT_OF_PAY'].apply(utils.fix_unitOfPay)

In [40]:
data['WAGE_UNIT_OF_PAY'].unique()

array(['Y', 'H', 'M', 'B', 'W', None], dtype=object)

In [41]:
data['PW_UNIT_OF_PAY'].unique()

array(['Year', 'Hour', nan, 'Month', 'Bi-Weekly', 'Week'], dtype=object)

In [42]:
data['PW_UNIT_OF_PAY'] = data['PW_UNIT_OF_PAY'].apply(utils.fix_unitOfPay)

In [43]:
data['PW_UNIT_OF_PAY'].unique()

array(['Y', 'H', None, 'M', 'B', 'W'], dtype=object)

In [ ]:
def calc_wage(x):
    wf = x.WAGE_RATE_OF_PAY_FROM
    wt = x.WAGE_RATE_OF_PAY_TO
    if pd.isnull(wt):
        return wf

    if wf <=wt:
        return (wf + wt)/2
    else:
        return wf

In [44]:
data['WAGE_RATE_OF_PAY_FROM'] = data['WAGE_RATE_OF_PAY_FROM'].replace(0, np.nan)
data['WAGE_RATE_OF_PAY_TO'] = data['WAGE_RATE_OF_PAY_TO'].replace(0, np.nan)
data['WAGE_RATE_OF_PAY'] = data[['WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO']].apply(utils.calc_wage, axis =1)

In [ ]:
data.head(30)

In [46]:
#LOOK AT YO CLEAN DATA
data.WAGE_RATE_OF_PAY_FROM.isnull().sum()

2

In [48]:
#Select only the columns that your data has that are in the master list
cdata = data[['CASE_SUBMITTED', 'CASE_NUMBER', 'CASE_STATUS', 'DECISION_DATE', 'VISA_CLASS',
    'JOB_TITLE', 'DOT_CODE', 'SOC_CODE', 'SOC_NAME', 'FULL_TIME_POSITION',
    'EMPLOYER_NAME', 'EMPLOYER_ADDRESS', 'EMPLOYER_CITY', 'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE',
    'EMPLOYER_AREA_CODE', 'NAICS_CODE', 'TOTAL_WORKERS', 
    'WORKSITE_CITY', 'WORKSITE_STATE', 'WORKSITE_POSTAL_CODE', 
    'WAGE_RATE_OF_PAY', 'WAGE_UNIT_OF_PAY', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY']]

In [49]:
cdata.head()

,CASE_SUBMITTED,CASE_NUMBER,CASE_STATUS,DECISION_DATE,VISA_CLASS,JOB_TITLE,DOT_CODE,SOC_CODE,SOC_NAME,FULL_TIME_POSITION,EMPLOYER_NAME,EMPLOYER_ADDRESS,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_AREA_CODE,NAICS_CODE,TOTAL_WORKERS,WORKSITE_CITY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,WAGE_RATE_OF_PAY,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY
0,2016-02-24,I-200-16055-173457,CERTIFIED-WITHDRAWN,2016-10-01,H-1B,ASSOCIATE DATA INTEGRATION,None,15-1121,COMPUTER SYSTEMS ANALYSTS,Y,DISCOVER PRODUCTS INC.,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,224,522210.0,1,RIVERWOODS,IL,60015,66565.5,Y,59197.0,Y
1,2016-03-04,I-200-16064-557834,CERTIFIED-WITHDRAWN,2016-10-01,H-1B,SENIOR ASSOCIATE,None,15-2031,OPERATIONS RESEARCH ANALYSTS,Y,DFS SERVICES LLC,2500 LAKE COOK ROAD,RIVERWOODS,IL,60015,224,522210.0,1,RIVERWOODS,IL,60015,55100.0,Y,49800.0,Y
2,2016-03-10,I-200-16063-996093,CERTIFIED-WITHDRAWN,2016-10-01,H-1B,NET SOFTWARE PROGRAMMER,None,15-1131,COMPUTER PROGRAMMERS,Y,EASTBANC TECHNOLOGIES LLC,1211 31ST ST NW,WASHINGTON,DC,20007,202,541511.0,2,WASHINGTON,DC,20007,77000.0,Y,76502.0,Y
3,2016-09-28,I-200-16272-196340,WITHDRAWN,2016-10-01,H-1B,PROJECT MANAGER,None,15-1199,"COMPUTER OCCUPATIONS, ALL OTHER",Y,INFO SERVICES LLC,17177 NORTH LAUREL PARK DR,LIVONIA,MI,48152,734,541511.0,1,JERSEY CITY,NJ,07302,102000.0,Y,90376.0,Y
4,2015-02-22,I-200-15053-636744,CERTIFIED-WITHDRAWN,2016-10-02,H-1B,ASSOCIATE ESOTERIC ASSET BACKED SECURITIES,None,13-2041,CREDIT ANALYSTS,Y,BB&T CORPORATION,223 WEST NASH STREET,WILSON,NC,27893,252,522110.0,1,NEW YORK,NY,10036,132500.0,Y,116605.0,Y


In [50]:
#Write out your clean data to a file

if not os.path.exists('cleaned_data'):
        os.mkdir('cleaned_data')
        
new_filename="2017.csv"
cdata.to_csv("cleaned_data/{}".format(new_filename), encoding='utf-8')